In [2]:
# Tweepy Machine

In [3]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import tweepy
import time
import os

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [4]:
# Loading Keys
api_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath('__file__')))))
file_name = os.path.join(api_dir, "api_keys.json")
data = json.load(open(file_name))

# Setting Keys
consumer_key = data['twitter_consumer_key']
consumer_secret = data['twitter_consumer_secret']
access_token = data['twitter_access_token']
access_token_secret = data['twitter_access_token_secret']

In [5]:
# Setting up Auth
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [6]:
# Runs Vader Analysis on Tweets based on a tag passed in
def runVader(target_tag):
    
    # Setting variables
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []
    date_list = []
    tweet_text_list = []
    
    # Loop through pages based on total count (20 tweets per page)
    for x in range(int(tweet_analyze_count/20)):

        # Get all tweets from home feed  --  Page=x was giving double 1st page results
        public_tweets = api.user_timeline(target_tag, page=x+1)

        # Loop through all tweets 
        for tweet in public_tweets:
            
            # Run Vader Analysis on each tweet and add values to the appropriate arrays
            compound_list.append(analyzer.polarity_scores(tweet["text"])["compound"])
            positive_list.append(analyzer.polarity_scores(tweet["text"])["pos"])
            negative_list.append(analyzer.polarity_scores(tweet["text"])["neg"])
            neutral_list.append(analyzer.polarity_scores(tweet["text"])["neu"])
            date_list.append(tweet["created_at"])
            tweet_text_list.append(tweet["text"])
            
    # Pushing results to a dict       
    sentiment = {
                "Date": date_list,
                "Compound": compound_list,
                "Positive": positive_list,
                "Neutral": negative_list,
                "Negative": neutral_list,
                "Text": tweet_text_list
    }
    
    # Creating a DF for results
    result = pd.DataFrame(sentiment)
    
    # Optional - Prints progress of API Calls
    #print('--------------------------------------------------------')
    #print(f'Processing Twitter handle: {target_tag}')
    #print('--------------------------------------------------------')
    
    # Returning a DF of the results
    return(result)

In [7]:
# Setting up function to call so sleep can be put inside a wait loop on a timer
def runTweeter(bot_acc):
    
    # Grabbing only most recent tweet - No queueing system
    mentions = api.home_timeline(count=1)
    
    # Clearing target tag
    target_tag = ""

    # Creating a try "loop" to check if the most recent status is valid
    try:
        target_tag = mentions[0]['entities']['user_mentions'][1]['screen_name']
        
    # Reseting target_tag if it fails
    except:
        target_tag = ""

    # Starting IF block to grab rest of info if needed
    if (mentions[0]['text'] == f'{bot_acc} Analyze: @{target_tag}'):

        # Grab who requested for tweet-back
        req_user = mentions[0]['user']['screen_name']

        # Optional - Confirm target_account
        #print("Target Account: " + target_tag)
        #print("Requesting User: " + req_user)

    # Return both users
    return(target_tag, req_user)   